
Importing the Libraries

In [ ]:
import os 
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot
import sklearn

Reading Dataset from csv file

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df=pd.read_csv('/drive/facial_expression/fer2013.csv')

FileNotFoundError: ignored


Printing First 5 values of the csv file

In [ ]:
df.head()

Printing Unique Emotion values

In [ ]:
df.emotion.unique()

Assigning a label to each value

In [ ]:
label_to_text={0:'anger', 1:'disgust', 2:'fear', 3:'happiness', 4:'sadness', 5:'surprise', 6:'neutral'}

Converting into Numpy matrix

In [ ]:
np.array(df.pixels.loc[0].split(' ')).reshape(48,48)

Visualize sample dataset

In [ ]:
fig=pyplot.figure(1,(14,14))
k=0
for label in sorted(df.emotion.unique()):
  for j in range(3):
    px=df[df.emotion==label].pixels.iloc[k]
    px=np.array(px.split(' ')).reshape(48,48).astype('float32')
    k+=1
    ax=pyplot.subplot(7,7,k)
    ax.imshow(px)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title(label_to_text[label])
    pyplot.tight_layout()

Reshaping Entire Data

In [ ]:
img_array=df.pixels.apply(lambda x : np.array(x.split(' ')).reshape(48,48,1).astype('float32'))

Printing the Shape

In [ ]:
img_array.shape

In [ ]:
img_array=np.stack(img_array,axis=0)

In [ ]:
labels=df.emotion.values

Importing Libraries

In [ ]:
from sklearn.model_selection import train_test_split

Splitting the Dataset into Training and Testing

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(img_array,labels,test_size=0.3)

Printing the shapes

In [ ]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

In [ ]:
X_train=X_train/255
X_test=X_test/255

Building the model

In [ ]:
basemodel=tf.keras.models.Sequential([tf.keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(48,48,1)),
                                      tf.keras.layers.MaxPool2D(2,2),
                                      
                                      tf.keras.layers.Conv2D(64,(3,3),activation='relu',input_shape=(48,48,1)),
                                      tf.keras.layers.MaxPool2D(2,2),
                                      
                                      tf.keras.layers.Conv2D(128,(3,3),activation='relu'),
                                      tf.keras.layers.MaxPool2D(2,2),
                                      
                                      tf.keras.layers.Conv2D(256,(3,3),activation='relu'),
                                      tf.keras.layers.MaxPool2D(2,2),
                                      tf.keras.layers.Flatten(),
                                      tf.keras.layers.Dense(1000,activation='relu'),
                                      tf.keras.layers.Dense(7,activation='softmax')
                                      ])


In [ ]:
basemodel.summary()

In [ ]:
basemodel.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [ ]:
import os
try:
  os.mkdir('checkpoint')
except:
  pass

In [ ]:
checkpoint="/content/drive/MyDrive/facial_expression/checkpoint"

Callback and checkpoint code

In [ ]:
filename='bestmodel.h5'
checkpoint_path=os.path.join(checkpoint,filename)
call_back=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                             monitor='val_accuracy',
                                             verbose=1,
                                             save_freq='epoch',
                                             save_best_only=True,
                                             save_weights_only=False,
                                             mode='max')

Fitting the model

In [ ]:
from tensorflow.keras.models import Model, load_model
history=basemodel.fit(X_train, y_train,
          batch_size=40,
          epochs=50,
          verbose=1,
          callbacks=call_back,
          validation_data=(X_test, y_test))

Result

In [ ]:
final_model=tf.keras.models.load_model(checkpoint_path)
from IPython.display import clear_output
import time
for k in range(40):
  print(f'actual label is {label_to_text[y_test[k]]}')
  predicted_class=final_model.predict(tf.expand_dims(X_test[k],0)).argmax()
  print(f'predicted label is {label_to_text[predicted_class]}')
  pyplot.imshow(X_test[k].reshape((48,48)))
  pyplot.show()
  time.sleep(3)
  clear_output(wait=True) 
